In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import subprocess
import pandas as pd
from datetime import date, timedelta
from io import StringIO
import time
subprocess.Popen([
    "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome",
    "--remote-debugging-port=9222",
    '--user-data-dir=/tmp/chrome-debug' 
])

option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")  # 디버깅 포트 연결

# WebDriver 생성
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)

기존 브라우저 세션에서 여는 중입니다.


In [30]:
start_date = date(2024,1,1)
end_date = date(2024,1,31)
city = "us/ny/new-york-city/KLGA"
base_url = f"https://www.wunderground.com/history/daily/{city}/date/"
res = []
def get_weather_period():
    curr_date = start_date
    while curr_date <= end_date:
        print(curr_date)
        url = base_url + f'{curr_date}'
        driver.get(url)
        WebDriverWait(driver,20)
        time.sleep(6)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        df_list = pd.read_html(StringIO(str(soup)))
        daily_weather_df = df_list[1].dropna()
        daily_weather_df.loc[:,'Date'] = f'{curr_date}'
        res.append(daily_weather_df)
        curr_date += timedelta(days=1)

In [ ]:
get_weather_period()

In [55]:
month_df = pd.concat(res,ignore_index=True)
month_df_modified = month_df.copy()

In [56]:
month_df.head()

,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,Date
0,12:51 AM,43 °F,27 °F,53 °%,SW,7 °mph,0 °mph,29.98 °in,0.0 °in,Cloudy,2024-01-01
1,1:51 AM,43 °F,27 °F,53 °%,W,9 °mph,0 °mph,29.97 °in,0.0 °in,Cloudy,2024-01-01
2,2:51 AM,42 °F,28 °F,58 °%,W,9 °mph,0 °mph,29.97 °in,0.0 °in,Cloudy,2024-01-01
3,3:51 AM,42 °F,29 °F,60 °%,W,7 °mph,0 °mph,29.96 °in,0.0 °in,Cloudy,2024-01-01
4,4:51 AM,42 °F,29 °F,60 °%,WSW,5 °mph,0 °mph,29.96 °in,0.0 °in,Cloudy,2024-01-01


In [ ]:
month_df_modified["Datetime"] = pd.to_datetime(month_df["Date"] + " " + month_df["Time"])
month_df_modified['Temperature'] = month_df['Temperature'].str[:-2].astype(int)
month_df_modified['Humidity'] = month_df['Humidity'].str[:-2].astype(int)
month_df_modified['Wind Speed'] = month_df['Wind Speed'].str[:-4].astype(int)
month_df_modified

/var/folders/05/2lrg20wj3k1dfc7p1z7789w40000gn/T/ipykernel_3531/3758071343.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  month_df_modified["Datetime"] = pd.to_datetime(month_df["Date"] + " " + month_df["Time"])


,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,Date,DateTime,Datetime
0,12:51 AM,43,27 °F,53,SW,7,0 °mph,29.98 °in,0.0 °in,Cloudy,2024-01-01,2025-01-30 00:51:00,2024-01-01 00:51:00
1,1:51 AM,43,27 °F,53,W,9,0 °mph,29.97 °in,0.0 °in,Cloudy,2024-01-01,2025-01-30 01:51:00,2024-01-01 01:51:00
2,2:51 AM,42,28 °F,58,W,9,0 °mph,29.97 °in,0.0 °in,Cloudy,2024-01-01,2025-01-30 02:51:00,2024-01-01 02:51:00
3,3:51 AM,42,29 °F,60,W,7,0 °mph,29.96 °in,0.0 °in,Cloudy,2024-01-01,2025-01-30 03:51:00,2024-01-01 03:51:00
4,4:51 AM,42,29 °F,60,WSW,5,0 °mph,29.96 °in,0.0 °in,Cloudy,2024-01-01,2025-01-30 04:51:00,2024-01-01 04:51:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,8:49 PM,39,28 °F,65,CALM,0,0 °mph,30.06 °in,0.0 °in,Cloudy,2024-01-31,2025-01-30 20:49:00,2024-01-31 20:49:00
934,8:51 PM,40,29 °F,65,CALM,0,0 °mph,30.06 °in,0.0 °in,Cloudy,2024-01-31,2025-01-30 20:51:00,2024-01-31 20:51:00
935,9:51 PM,39,30 °F,70,NW,5,0 °mph,30.06 °in,0.0 °in,Mostly Cloudy,2024-01-31,2025-01-30 21:51:00,2024-01-31 21:51:00
936,10:51 PM,39,30 °F,70,NW,3,0 °mph,30.06 °in,0.0 °in,Cloudy,2024-01-31,2025-01-30 22:51:00,2024-01-31 22:51:00


In [70]:
result_df = month_df_modified[['Datetime','Temperature','Humidity','Wind Speed','Condition']]
result_df

,Datetime,Temperature,Humidity,Wind Speed,Condition
0,2024-01-01 00:51:00,43,53,7,Cloudy
1,2024-01-01 01:51:00,43,53,9,Cloudy
2,2024-01-01 02:51:00,42,58,9,Cloudy
3,2024-01-01 03:51:00,42,60,7,Cloudy
4,2024-01-01 04:51:00,42,60,5,Cloudy
...,...,...,...,...,...
933,2024-01-31 20:49:00,39,65,0,Cloudy
934,2024-01-31 20:51:00,40,65,0,Cloudy
935,2024-01-31 21:51:00,39,70,5,Mostly Cloudy
936,2024-01-31 22:51:00,39,70,3,Cloudy


In [72]:
result_df.to_csv('weather.csv',index=False)